In [ ]:
import numpy as np
import cv2
import os
from keras._tf_keras.keras.preprocessing import image
from skimage.feature import local_binary_pattern

In [ ]:
def extract_lbp_features(images, P=24, R=3):
    lbp_features = []
    for img in images:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        lbp = local_binary_pattern(gray, P=P, R=R, method="uniform")
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, P + 3), range=(0, P + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-7)  # Normalize histogram
        lbp_features.append(hist)
    return np.array(lbp_features)

In [ ]:
def preprocess_image(paths, target_size=(224, 224)):
    processed_img_paths = []
    for image_path in paths:
        img = image.load_img(image_path, target_size=target_size)
        img = image.img_to_array(img).astype(np.uint8)  # Convert to uint8 for LBP
        processed_img_paths.append(img)
    return np.array(processed_img_paths)

In [ ]:
def extract_lbp_features_from_directory(directory_path, output_file_name):
    categories = {"male": 0, "female": 1}

    image_paths = []
    labels = []

    for category, label in categories.items():
        category_path = os.path.join(directory_path, category)
        for img_name in sorted(os.listdir(category_path)):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                img_path = os.path.join(category_path, img_name)    
                image_paths.append(img_path)
                labels.append(label)

    processed_images = preprocess_image(paths=image_paths)
    features_array = extract_lbp_features(processed_images)
    labels_array = np.array(labels)

    np.savez(f'{output_file_name}.npz', features=features_array, labels=labels_array)


In [ ]:
# scheme 1
extract_lbp_features_from_directory(directory_path="../dataset/scheme-1", output_file_name="lbp_1")

# scheme 2
extract_lbp_features_from_directory(directory_path="../dataset/scheme-2", output_file_name="lbp_2")